<a href="https://colab.research.google.com/github/AlexMan2000/DS-GA-1003-Final-Repo/blob/main/ExplainableCNN_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Explainable CNN Project Code


This is the code base of our DS-GA-1003 Final Project.

# Basic Settings

Connect to google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
import os
os.makedirs('/content/gdrive/My Drive/NYU_DS_GA_1003_XAI', exist_ok=True)
os.chdir('/content/gdrive/My Drive/NYU_DS_GA_1003_XAI')
!ls

In [ ]:
# Clone this git-repo
!git clone "https://github.com/AlexMan2000/metrics-saliency-maps"

## Environment Settings

In [ ]:


# download and unzip training data
!gdown --id '1QntUQuWJoVR8h5FoeDa56xrQSdcCwFeD' --output food.zip
!unzip food.zip

In [ ]:
# download pretrained model
!gdown --id '1-Qw-oIJ0cSo2iG_n_U9mcJqXc2-LCSdV' --output checkpoint.pth

In [ ]:
# install lime in colab
!pip install lime==0.1.1.37

# Support for gradcam
!pip install captum

In [ ]:
import os
import sys
import argparse
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.utils.data import Dataset
import torchvision.transforms as transforms
from skimage.segmentation import slic
from skimage.segmentation import mark_boundaries
from lime import lime_image
from pdb import set_trace
from torch.autograd import Variable
from captum.attr import LayerGradCam,LayerAttribution

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
sys.path.append("/content/gdrive/My Drive/NYU_DS_GA_1003_XAI/metrics-saliency-maps/saliency_maps_metrics")
from single_step_metrics import IIC_AD, ADD
from multi_step_metrics import Deletion, Insertion

## Argument Parsing

In [ ]:
args = {
      'ckptpath': './checkpoint.pth',
      'dataset_dir': './food/'
}
args = argparse.Namespace(**args)

# Model and Dataset

## Model Definition and Checkpoint Loading

In [ ]:
# Model definition
class Classifier(nn.Module):
  def __init__(self):
    super(Classifier, self).__init__()

    def building_block(indim, outdim):
      return [
        nn.Conv2d(indim, outdim, 3, 1, 1),
        nn.BatchNorm2d(outdim),
        nn.ReLU(),
      ]
    def stack_blocks(indim, outdim, block_num):
      layers = building_block(indim, outdim)
      for i in range(block_num - 1):
        layers += building_block(outdim, outdim)
      layers.append(nn.MaxPool2d(2, 2, 0))
      return layers

    cnn_list = []
    cnn_list += stack_blocks(3, 128, 3)
    cnn_list += stack_blocks(128, 128, 3)
    cnn_list += stack_blocks(128, 256, 3)
    cnn_list += stack_blocks(256, 512, 1)
    cnn_list += stack_blocks(512, 512, 1)
    self.cnn = nn.Sequential( * cnn_list)

    dnn_list = [
      nn.Linear(512 * 4 * 4, 1024),
      nn.ReLU(),
      nn.Dropout(p = 0.3),
      nn.Linear(1024, 11),
    ]
    self.fc = nn.Sequential( * dnn_list)

  def forward(self, x):
    out = self.cnn(x)
    out = out.reshape(out.size()[0], -1)
    return self.fc(out)

In [ ]:
# Load trained model
model_cpu = Classifier()
model = Classifier().cuda()
checkpoint = torch.load(args.ckptpath)
model.load_state_dict(checkpoint['model_state_dict'])
# It should display: <All keys matched successfully>

## Dataset Definition and Creation

In [ ]:
# It might take some time, if it is too long, try to reload it.
# Dataset definition
class FoodDataset(Dataset):
    def __init__(self, paths, labels, mode):
        # mode: 'train' or 'eval'

        self.paths = paths
        self.labels = labels
        trainTransform = transforms.Compose([
            transforms.Resize(size=(128, 128)),
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(15),
            transforms.ToTensor(),
        ])
        evalTransform = transforms.Compose([
            transforms.Resize(size=(128, 128)),
            transforms.ToTensor(),
        ])
        self.transform = trainTransform if mode == 'train' else evalTransform

    # pytorch dataset class
    def __len__(self):
        return len(self.paths)

    def __getitem__(self, index):
        X = Image.open(self.paths[index])
        X = self.transform(X)
        Y = self.labels[index]
        return X, Y

    # help to get images for visualizing
    def getbatch(self, indices):
        images = []
        labels = []
        for index in indices:
          image, label = self.__getitem__(index)
          images.append(image)
          labels.append(label)
        return torch.stack(images), torch.tensor(labels)

# help to get data path and label
def get_paths_labels(path):
    def my_key(name):
      return int(name.replace(".jpg","").split("_")[1])+1000000*int(name.split("_")[0])
    imgnames = os.listdir(path)
    imgnames.sort(key=my_key)
    imgpaths = []
    labels = []
    for name in imgnames:
        imgpaths.append(os.path.join(path, name))
        labels.append(int(name.split('_')[0]))
    return imgpaths, labels
train_paths, train_labels = get_paths_labels(args.dataset_dir)

train_set = FoodDataset(train_paths, train_labels, mode='eval')

## The Images for Observation
There are 11 categories of food: Bread, Dairy product, Dessert, Egg, Fried food, Meat, Noodles/Pasta, Rice, Seafood, Soup, and Vegetable/Fruit.
The images are marked from 0 to 9.

In [ ]:
img_indices = [i for i in range(10)]
images, labels = train_set.getbatch(img_indices)
fig, axs = plt.subplots(1, len(img_indices), figsize=(15, 8))
for i, img in enumerate(images):
  axs[i].imshow(img.cpu().permute(1, 2, 0))
# print(labels) # this line can help you know the labels of each image

In [ ]:
images.shape

# Preliminaries: What are we doing?
Basically we want to evaluate the faithfulness(e.g. reliability) of the CNN explanation methods(e.g. Saliency Map, GradCam).


Faithfulness means to what extent I can trust saliency map to explain my CNN model. Does its explanation aligns with my judgement.

The image below shows the gist of our project.



![](https://github.com/TristanGomez44/metrics-saliency-maps/raw/main/pics/metrics_repo_illust.png)

## What are those XAI Algorithms?

Those XAI Algorithms are responsible for explaining what a CNN model has learned in the image classification task.



For example in the image above.
- As a human being, I will say since I see the bird in the image, so I classify the image as bird. From my perspective, something in the image that looks like a "bird" makes me think and classify it as a bird.
- As a machine learning model, it may think that  I learn some features(e.g. model parameters like kernel filters $W$ or bias $b$) from the image that makes me classify it as a bird. For example, since the model sees certain pixels in the image(e.g. pixels around bird's beak), it classifies this image as a bird.


So a model must have "see" some portion of the image so that it can correctly classify an input image.

Those algorithms are responsible for finding what on earth the model is "see"ing.


Algorithm ideas are detailed in XAI Part 1.


## How to evaluate?

For those XAI Algorithms, we will have a set of evaluation metrics(e.g. DAUC, IAUC, DC, IC) that aims to quantify their performance.

## Terminology

- Explanation Method:
- Heatmap:
- Saliency Map



# **XAI Part 1: CNN Heatmap XAI Algorithms**

## Lime Package
[Lime](https://github.com/marcotcr/lime) is a package about explaining what machine learning classifiers are doing. We can first use it to observe the model.

In [ ]:
def predict(input):
    # input: numpy array, (batches, height, width, channels)

    model.eval()
    input = torch.FloatTensor(input).permute(0, 3, 1, 2)
    # pytorch tensor, (batches, channels, height, width)

    output = model(input.cuda())
    return output.detach().cpu().numpy()

def segmentation(input):
    # split the image into 200 pieces with the help of segmentaion from skimage
    return slic(input, n_segments=200, compactness=1, sigma=1, start_label=1)


fig, axs = plt.subplots(1, len(img_indices), figsize=(15, 8))
# fix the random seed to make it reproducible
np.random.seed(16)
for idx, (image, label) in enumerate(zip(images.permute(0, 2, 3, 1).numpy(), labels)):
    x = image.astype(np.double)
    # numpy array for lime

    explainer = lime_image.LimeImageExplainer()
    explaination = explainer.explain_instance(image=x, classifier_fn=predict, segmentation_fn=segmentation)

    # doc: https://lime-ml.readthedocs.io/en/latest/lime.html?highlight=explain_instance#lime.lime_image.LimeImageExplainer.explain_instance

    lime_img, mask = explaination.get_image_and_mask(
                                label=label.item(),
                                positive_only=False,
                                hide_rest=False,
                                num_features=11,
                                min_weight=0.05
                            )
    # turn the result from explainer to the image
    # doc: https://lime-ml.readthedocs.io/en/latest/lime.html?highlight=get_image_and_mask#lime.lime_image.ImageExplanation.get_image_and_mask

    axs[idx].imshow(lime_img)

plt.show()
plt.close()

## Saliency Map(Baseline)

What is Saliency map ?

The heatmaps that highlight pixels of the input image that contribute the most in the classification task.

Ref: https://medium.com/datadriveninvestor/visualizing-neural-networks-using-saliency-maps-in-pytorch-289d8e244ab4

We put an image into the model, forward then calculate the loss referring to the label. Therefore, the loss is related to:


*   image
*   model parameters
*   label

Generally speaking, we change model parameters to fit "image" and "label". When backward, we calculate the partial differential value of **loss to model parameters**.

Now, we have another look. When we change the image's pixel value, the partial differential value of **loss to image** shows the change in the loss. We can say that it means the importance of the pixel. We can visualize it to demonstrate which part of the image contribute the most to the model's judgment.

In [ ]:
def normalize(image):
  return (image - image.min()) / (image.max() - image.min())
  # return torch.log(image)/torch.log(image.max())

def compute_saliency_maps(x, y, model):
  model.eval()
  x = x.cuda()

  # we want the gradient of the input x
  x.requires_grad_()


  y_pred = model(x)


  loss_func = torch.nn.CrossEntropyLoss()
  loss = loss_func(y_pred, y.cuda())

  loss.backward()

  # saliencies = x.grad.abs().detach().cpu()

  # Saliency would be the gradient with respect to the input image now. But note that the input image has 3 channels,
  # R, G and B. To derive a single class saliency value for each pixel (i, j),  we take the maximum magnitude
  # across all colour channels.
  saliencies, _ = torch.max(x.grad.data.abs().detach().cpu(),dim=1)

  # We need to normalize each image, because their gradients might vary in scale
  saliencies = torch.stack([normalize(item) for item in saliencies])
  return saliencies

In [ ]:
# images, labels = train_set.getbatch(img_indices)
saliencies = compute_saliency_maps(images, labels, model)

# visualize
fig, axs = plt.subplots(2, len(img_indices), figsize=(15, 8))
for row, target in enumerate([images, saliencies]):
  for column, img in enumerate(target):
    if row==0:
      axs[row][column].imshow(img.permute(1, 2, 0).numpy())
      # What is permute?
      # In pytorch, the meaning of each dimension of image tensor is (channels, height, width)
      # In matplotlib, the meaning of each dimension of image tensor is (height, width, channels)
      # permute is a tool for permuting dimensions of tensors
      # For example, img.permute(1, 2, 0) means that,
      # - 0 dimension is the 1 dimension of the original tensor, which is height
      # - 1 dimension is the 2 dimension of the original tensor, which is width
      # - 2 dimension is the 0 dimension of the original tensor, which is channels
    else:
      axs[row][column].imshow(img.numpy(), cmap=plt.cm.hot)

plt.show()
plt.close()

## Smooth Grad

The method of Smooth grad is to randomly add noise to the image and get different heatmaps. The average of the heatmaps would be more robust to noisy gradient.

ref: https://arxiv.org/pdf/1706.03825.pdf

SmoothGrad works in the following way:
1. Generate multiple versions of the image of interest by adding noise to it.
2. Create pixel attribution maps for all images.
3. Average the pixel attribution maps.

Yes, it is that simple. Why should this work? The theory is that the derivative fluctuates greatly at small scales. Neural networks have no incentive during training to keep the gradients smooth, their goal is to classify images correctly. Averaging over multiple maps "smooths out" these fluctuations:
$$
R_{s g}(x)=\frac{1}{N} \sum_{i=1}^n R\left(x+g_i\right),
$$

Here, $g_i \sim N\left(0, \sigma^2\right)$ are noise vectors sampled from the Gaussian distribution. The "ideal" noise level depends on the input image and the network. The authors suggest a noise level of $10 \%-20 \%$, which means that $\frac{\sigma}{x_{\max }-x_{\min }}$ should be between 0.1 and 0.2 . The limits $x_{\min }$ and $x_{\max }$ refer to minimum and maximum pixel values of the image. The other parameter is the number of samples $n$, for which was suggested to use $n=50$, since there are diminishing returns above that.

In [ ]:
# Smooth grad

def normalize(image):
  return (image - image.min()) / (image.max() - image.min())

def smooth_grad(x, y, model, epoch, param_sigma_multiplier):
  model.eval()
  # x = x.cuda().unsqueeze(0)

  mean = 0
  sigma = param_sigma_multiplier / (torch.max(x) - torch.min(x)).item()
  smooth = np.zeros(x.cuda().unsqueeze(0).size())
  for i in range(epoch):
    # call Variable to generate random noise
    noise = Variable(x.data.new(x.size()).normal_(mean, sigma**2))
    x_mod = (x+noise).unsqueeze(0).cuda()
    x_mod.requires_grad_()

    y_pred = model(x_mod)
    loss_func = torch.nn.CrossEntropyLoss()
    loss = loss_func(y_pred, y.cuda().unsqueeze(0))
    loss.backward()

    # like the method in saliency map
    smooth += x_mod.grad.abs().detach().cpu().data.numpy()

  smooth = normalize(smooth / epoch) # don't forget to normalize


  # Maximum value across all channels to obtain a singla value.
  torch_smooth = torch.max(torch.tensor(smooth, dtype=torch.float32), dim=1)
  smooth = torch_smooth[0].numpy()
  # smooth = smooth / epoch
  return smooth

def trigger():
  # images, labels = train_set.getbatch(img_indices)
  smooth = []
  for i, l in zip(images, labels):
    smooth.append(smooth_grad(i, l, model, 500, 0.4))
  smooth = np.stack(smooth)
  # print(smooth.shape)

  fig, axs = plt.subplots(2, len(img_indices), figsize=(15, 8))
  for row, target in enumerate([images, smooth]):
    for column, img in enumerate(target):
      if row==0:

        axs[row][column].imshow(img.permute(1, 2, 0).numpy())
      else:
        axs[row][column].imshow(img.transpose((1,2,0)), cmap=plt.cm.hot)

  return smooth

smooth = trigger()

## Filter Explanation

In this part, we want to know what a specific filter recognize, we'll do
- Filter activation: pick up some images, and check which part of the image activates the filter
- Filter visualization: look for which kind of image can activate the filter the most


The problem is that, in normal case, we'll directly feed the image to the model, for example,
```
loss = model(image)
loss.backward()
```

How can we get the output of a specific layer of CNN?

We can modify the model definition, make the forward function not only return loss but also retrun the activation map. But this is difficult to maintain the code. As a result, pytorch offers a better solution: **hook**


In [ ]:
def normalize(image):
  return (image - image.min()) / (image.max() - image.min())

layer_activations = None
def filter_explanation(x, model, cnnid, filterid, iteration=100, lr=1):
  # x: input image
  # cnnid: cnn layer id
  # filterid: which filter
  model.eval()

  def hook(model, input, output):
    global layer_activations
    layer_activations = output

  hook_handle = model.cnn[cnnid].register_forward_hook(hook)
  # When the model forwards through the layer[cnnid], it needs to call the hook function first
  # The hook function save the output of the layer[cnnid]
  # After forwarding, we'll have the loss and the layer activation

  # Filter activation: x passing the filter will generate the activation map
  model(x.cuda()) # forward

  # Based on the filterid given by the function argument, pick up the specific filter's activation map
  # We just need to plot it, so we can detach from graph and save as cpu tensor
  filter_activations = layer_activations[:, filterid, :, :].detach().cpu()

  # Filter visualization: find the image that can activate the filter the most
  x = x.cuda()
  x.requires_grad_()
  # input image gradient
  optimizer = Adam([x], lr=lr)
  # Use optimizer to modify the input image to amplify filter activation
  for iter in range(iteration):
    optimizer.zero_grad()
    model(x)

    objective = -layer_activations[:, filterid, :, :].sum()
    # We want to maximize the filter activation's summation
    # So we add a negative sign

    objective.backward()
    # Calculate the partial differential value of filter activation to input image
    optimizer.step()
    # Modify input image to maximize filter activation
  filter_visualizations = x.detach().cpu().squeeze()

  # Don't forget to remove the hook
  hook_handle.remove()
  # The hook will exist after the model register it, so you have to remove it after used
  # Just register a new hook if you want to use it

  return filter_activations, filter_visualizations

In [ ]:
images, labels = train_set.getbatch(img_indices)
filter_activations, filter_visualizations = filter_explanation(images, model, cnnid=6, filterid=0, iteration=100, lr=0.1)

fig, axs = plt.subplots(3, len(img_indices), figsize=(15, 8))
for i, img in enumerate(images):
  axs[0][i].imshow(img.permute(1, 2, 0))
# Plot filter activations
for i, img in enumerate(filter_activations):
  axs[1][i].imshow(normalize(img))
# Plot filter visualization
for i, img in enumerate(filter_visualizations):
  axs[2][i].imshow(normalize(img.permute(1, 2, 0)))
plt.show()
plt.close()

In [ ]:
images, labels = train_set.getbatch(img_indices)
filter_activations, filter_visualizations = filter_explanation(images, model, cnnid=23, filterid=0, iteration=100, lr=0.1)

# Plot filter activations
fig, axs = plt.subplots(3, len(img_indices), figsize=(15, 8))
for i, img in enumerate(images):
  axs[0][i].imshow(img.permute(1, 2, 0))
for i, img in enumerate(filter_activations):
  axs[1][i].imshow(normalize(img))
for i, img in enumerate(filter_visualizations):
  axs[2][i].imshow(normalize(img.permute(1, 2, 0)))
plt.show()
plt.close()


##Integrated Gradients
https://arxiv.org/pdf/1703.01365.pdf


In [ ]:
class IntegratedGradients():
    def __init__(self, model):
        self.model = model
        self.gradients = None
        # Put model in evaluation mode
        self.model.eval()

    def generate_images_on_linear_path(self, input_image, steps):
        # Generate scaled xbar images
        xbar_list = [input_image*step/steps for step in range(steps)]
        return xbar_list

    def generate_gradients(self, input_image, target_class):
        # We want to get the gradients of the input image
        input_image.requires_grad=True
        # Forward
        model_output = self.model(input_image)
        # Zero grads
        self.model.zero_grad()
        # Target for backprop
        one_hot_output = torch.FloatTensor(1, model_output.size()[-1]).zero_().cuda()
        one_hot_output[0][target_class] = 1
        # Backward
        model_output.backward(gradient=one_hot_output)
        self.gradients = input_image.grad
        # Convert Pytorch variable to numpy array
        # [0] to get rid of the first channel (1,3,128,128)
        gradients_as_arr = self.gradients.data.cpu().numpy()[0]
        return gradients_as_arr

    def generate_integrated_gradients(self, input_image, target_class, steps):
        # Generate xbar images
        xbar_list = self.generate_images_on_linear_path(input_image, steps)
        # Initialize an image composed of zeros
        integrated_grads = np.zeros(input_image.size())
        for xbar_image in xbar_list:
            # Generate gradients from xbar images
            single_integrated_grad = self.generate_gradients(xbar_image, target_class)
            # Add rescaled grads from xbar images
            integrated_grads = integrated_grads + single_integrated_grad/steps
        # [0] to get rid of the first channel (1,3,128,128)


        return integrated_grads[0]

def normalize(image):
  return (image - image.min()) / (image.max() - image.min())

In [ ]:
# put the image to cuda
images, labels = train_set.getbatch(img_indices)
images = images.cuda()

IG = IntegratedGradients(model)
integrated_grads = []
for i, img in enumerate(images):
  img = img.unsqueeze(0)
  integrated_grads.append(IG.generate_integrated_gradients(img, labels[i], 10))

fig, axs = plt.subplots(2, len(img_indices), figsize=(15, 8))
for i, img in enumerate(images):
  axs[0][i].imshow(img.cpu().permute(1, 2, 0))
for i, img in enumerate(integrated_grads):
  axs[1][i].imshow(np.moveaxis(normalize(img),0,-1))
plt.show()
plt.close()

## Grad-CAM
https://christophm.github.io/interpretable-ml-book/pixel-attribution.html#vanilla-gradient-saliency-maps

The following is the recipe for Grad-CAM. Our goal is to find the localization map, which is defined as:
$$
L_{\text {Grad-CAM }}^c \in \mathbb{R}^{u \times v}=\underbrace{\operatorname{ReLU}}_{\text {Pick positive values }}\left(\sum_k \alpha_k^c A^k\right)
$$

Here, $u$ is the width, $v$ the height of the explanation and $c$ the class of interest.
1. Forward-propagate the input image through the convolutional neural network.
2. Obtain the raw score for the class of interest, meaning the activation of the neuron before the softmax layer.
3. Set all other class activations to zero.
4. Back-propagate the gradient of the class of interest to the last convolutional layer before the fully connected layers: $\frac{\delta y^c}{\delta A^k}$.
5. Weight each feature map "pixel" by the gradient for the class. Indices $i$ and $j$ refer to the width and height dimensions:
$$
\alpha_k^c=\overbrace{\frac{1}{Z} \sum_i \sum_j}^{\text {global average pooling }} \underbrace{\frac{\delta y^c}{\delta A_{i j}^k}}_{\text {gradients via backprop }}
$$

This means that the gradients are globally pooled.
6. Calculate an average of the feature maps, weighted per pixel by the gradient.
7. Apply ReLU to the averaged feature map.
8. For visualization: Scale values to the interval between 0 and 1 . Upscale the image and overlay it over the original image.
9. Additional step for Guided Grad-CAM: Multiply heatmap with guided backpropagation.

In [ ]:
images, labels = train_set.getbatch(img_indices)

def compute_gradcam(x, y, cnnid, model):
  model = model.eval()
  x = x.cuda()

  layer_to_analyze = model.cnn[cnnid]
  gradcam = LayerGradCam(model, layer_to_analyze)

  # Step 1,2 Obtain relu output before softmax
  class_ind = model(x).argmax(dim=-1)

  # Step 3,4: Calculate the gradient to the layer's feature map specified by cnn[cnnid]
  attr = gradcam.attribute(x, class_ind)

  # Step 5~7: Upsample to the size of input image
  upsampled_attr = LayerAttribution.interpolate(attr, (128, 128))

  # Step 8 :Scale the value to between [0, 1]
  upsampled_attr = (upsampled_attr - upsampled_attr.min())/(upsampled_attr.max()-upsampled_attr.min())

  # Normalize the image
  x = (x - x.min())/(x.max()-x.min())

  # Apply the mask
  img_to_viz = (upsampled_attr * x.mean(dim=1,keepdim=True)).detach().cpu()

  return upsampled_attr, img_to_viz


def trigger():
  attr, img_to_viz = compute_gradcam(images, labels, 6, model)

  # Plot filter activations
  fig, axs = plt.subplots(2, len(img_indices), figsize=(15, 8))
  for i, img in enumerate(images):
    axs[0][i].imshow(img.permute(1, 2, 0))
  for i, img in enumerate(img_to_viz):
    axs[1][i].imshow(img.permute(1, 2, 0))

  plt.show()
  plt.close()

  return attr, img_to_viz

gradcam_expl, img_to_viz = trigger()

In [ ]:
model(images[0].unsqueeze(0).cuda())

## Organize the plot

In [ ]:


# for i, img in enumerate(img_to_viz):
#   axs[5, i].imshow(img.permute(1, 2, 0))

# XAI Part 2 Evaluations


## Helper Methods

In [ ]:
# Helper functions
def multi_step_ingredients(images, labels, model, algo_mode):

  assert algo_mode in ["Grad_CAM", "Saliency_Map", "Smooth_Grad", "Integrated_Grad"]

  allImg = []
  allExpl = []
  allInds = []

  for ind, img in enumerate(images):

      img = img.unsqueeze(0)
      class_ind = model(img.cuda()).argmax(dim=-1)

      # print("input image", img.shape)


      # Get the exaplanation map, should be of shape (1, 1, 128, 128) <-> (N, C, H, W)
      if algo_mode == "Grad_CAM":
        explanation,_ = compute_gradcam(img, class_ind, 6, model)
      elif algo_mode == "Saliency_Map":
        explanation = compute_saliency_maps(img, labels[ind].unsqueeze(0), model)
        explanation = explanation.unsqueeze(0)
      elif algo_mode == "Smooth_Grad":
        explanation = smooth_grad(img.squeeze(0), labels[ind], model, 500, 0.4)
        explanation = torch.tensor(explanation).unsqueeze(0)
        # print("expl", explanation.shape)
      elif algo_mode == "Integrated_Grad":
        IG = IntegratedGradients(model)
        # Output is (3, 128, 128) on numpy
        explanation = IG.generate_integrated_gradients(img.cuda(), labels[ind], 10)
        # Transform into a single gradient map as before (1, 1, 128, 128)
        explanation = torch.max(torch.tensor(explanation, dtype=torch.float32).unsqueeze(0), dim=1)[0].unsqueeze(0)

      allImg.append(img.cuda())
      allExpl.append(explanation)
      allInds.append(class_ind)

  allImg = torch.cat(allImg,dim=0)
  allExpl = torch.cat(allExpl,dim=0)
  allInds = torch.cat(allInds,dim=0)

  return allImg, allExpl, allInds


# Given a batch of images and their labels, evaluate the
# explanation methods(algo_mode) with a particular evaluation metric(eval_mode)
def computeMetrics(images, labels, model, algo_mode):
  allImg, allExpl, allInds = multi_step_ingredients(images, labels, model, algo_mode)

  deletion = Deletion()
  result_dic, dauc_list, dc_list = deletion(model,allImg.clone(),allExpl.clone(),allInds)
  dauc_mean = result_dic["dauc"]
  dc_mean = result_dic["dc"]

  insertion = Insertion()
  result_dic, iauc_list, ic_list = insertion(model,allImg.clone(),allExpl.clone(),allInds)
  iauc_mean = result_dic["iauc"]
  ic_mean = result_dic["ic"]


  iic_ad = IIC_AD()
  result_dic, iic_list, ad_list = iic_ad(model,allImg,allExpl,allInds)
  iic_mean,ad_mean = result_dic["iic"],result_dic["ad"]

  add = ADD()
  result_dic, add_list = add(model,allImg,allExpl,allInds)
  add_mean = result_dic["add"]


  result_dict = {
            "DAUC": dauc_mean
            ,"IAUC": iauc_mean
            ,"DC": dc_mean
            ,"IC":ic_mean
            ,"IIC": iic_mean
            ,"AD": ad_mean
            ,"ADD": add_mean
          }


  result_dict_detailed = {
            "DAUC": dauc_list
            ,"IAUC": iauc_list
            ,"DC": dc_list
            ,"IC": ic_list
            ,"IIC": iic_list
            ,"AD": ad_list
            ,"ADD": add_list
          }


  return result_dict, result_dict_detailed

In [ ]:
def temp():
  res = {"auc":[1,2,3], "ic":[2,23,4]}
  return 1, *list(res.values())
temp()

## Compute Evaluation

### Trigger Codes

In [ ]:
GradCAM_dict_simple, GradCAM_dict_detailed = computeMetrics(images, labels, model, "Grad_CAM")

In [ ]:
Saliency_Map_dict_simple, Saliency_Map_dict_detailed = computeMetrics(images, labels, model, "Saliency_Map")

In [ ]:
Smooth_Grad_dict_simple, Smooth_Grad_dict_detailed = computeMetrics(images, labels, model, "Smooth_Grad")

In [ ]:
Integrated_Grad_dict_simple, Integrated_Grad_dict_detailed = computeMetrics(images, labels, model, "Integrated_Grad")

In [ ]:
GradCAM_dict_detailed

In [ ]:
Integrated_Grad_dict_detailed

## Area-Based Metrics

### DAUC

The intuition behind this is that if a saliency map highlights
the areas that are relevant to the decision, masking them will result in a large
decrease of the initially predicted class score, which in turn will minimize the
AUC. **Therefore, minimizing this metric corresponds to an improvement.**

### IAUC

Instead of progressively masking the image, the IAUC metric starts from a
blurred image and then progressively unblurs it by starting from the most important areas according to the saliency map. Similarly, if the areas highlighted
by the map are relevant for predicting the correct category, the score of the corresponding class (obtained using the partially unblurred image) is supposed to
increase rapidly. **Conversely, maximizing this metric corresponds to an improvement.**

### Limitations

**DAUC and IAUC generate out of distribution (OOD) images**

When
progressively masking/unblurring the input image, the model is presented with
samples that can be considered out of the training distribution.

This kind of distortions produced by the masking/blurring operations
do not exist naturally in the dataset and are different from the kind produced
by the standard data augmentations like random crop, horizontal flip, and color
jitter, meaning that the model has not learned to process images with such
distortions. Therefore, the distribution of the images presented to the model is
different from the one met during training.

This shows that DAUC and IAUC may not reflect
the faithfulness of explanation methods as they are based on a behavior of the
model that is different from that encountered when facing training distribution

## Calibration-Based Metric

### Deletion Correlation(DC)

### Insertion Correlation(IC)

## Organize the Output

In [ ]:
fig, axs = plt.subplots(5, len(img_indices), figsize=(20, 12))
fig.tight_layout(pad=1.2)
for i, img in enumerate(images):
  if i == 5:
    axs[0, i].set_title("Original Image")
  axs[0, i].imshow(img.permute(1, 2, 0))
  axs[0, i].xaxis.set_visible(False)
  axs[0, i].yaxis.set_visible(False)

for i, img in enumerate(saliencies):
  if i == 5:
    axs[1, i].set_title("Saliency Map")
  axs[1, i].imshow(img.numpy(), cmap=plt.cm.hot)
  # axs[1, i].xaxis.set_visible(False)
  axs[1, i].yaxis.set_visible(False)
  axs[1, i].set_xlabel(f"DAUC:{round(Saliency_Map_dict_detailed['DAUC'][i], 2)}\n IAUC:{round(Saliency_Map_dict_detailed['IAUC'][i], 2)} \n DC:{round(Saliency_Map_dict_detailed['DC'][i], 2)}")



for i, img in enumerate(smooth):
  if i == 5:
    axs[2, i].set_title("Smooth Grad")
  axs[2, i].imshow(img.transpose((1,2,0)),cmap=plt.cm.hot)
  # axs[2, i].xaxis.set_visible(False)
  axs[2, i].yaxis.set_visible(False)
  axs[2, i].set_xlabel(f"DAUC:{round(Smooth_Grad_dict_detailed['DAUC'][i], 2)}\n IAUC:{round(Smooth_Grad_dict_detailed['IAUC'][i], 2)} \n DC:{round(Smooth_Grad_dict_detailed['DC'][i], 2)}")

for i, img in enumerate(integrated_grads):
  if i == 5:
    axs[3, i].set_title("Integrated Grad")
  axs[3, i].imshow(np.moveaxis(normalize(img),0,-1))
  # axs[3, i].xaxis.set_visible(False)
  axs[3, i].yaxis.set_visible(False)
  axs[3, i].set_xlabel(f"DAUC:{round(Integrated_Grad_dict_detailed['DAUC'][i], 2)}\n IAUC:{round(Integrated_Grad_dict_detailed['IAUC'][i], 2)} \n DC:{round(Integrated_Grad_dict_detailed['DC'][i], 2)}")

for i, img in enumerate(gradcam_expl):
  if i == 5:
    axs[4, i].set_title("Grad CAM")
  axs[4, i].imshow(img.detach().cpu().permute(1, 2, 0))
  # axs[4, i].xaxis.set_visible(False)
  axs[4, i].yaxis.set_visible(False)
  axs[4, i].set_xlabel(f"DAUC:{round(GradCAM_dict_detailed['DAUC'][i], 2)}\n IAUC:{round(GradCAM_dict_detailed['IAUC'][i], 2)} \n DC:{round(GradCAM_dict_detailed['DC'][i], 2)}")

# XAI Part 3 Grad_CAM Level Comparison


Explore Grad-CAM's explanation performance across different hidden layers.

In [ ]:
def get_GradCAMscore(images, labels, model, level_id):

  allImg = []
  allExpl = []
  allInds = []

  for ind, img in enumerate(images):

      img = img.unsqueeze(0)
      class_ind = model(img.cuda()).argmax(dim=-1)

      # Get the exaplanation map, should be of shape (1, 1, 128, 128) <-> (N, C, H, W)
      explanation,_ = compute_gradcam(img, class_ind, level_id, model)
      allImg.append(img.cuda())
      allExpl.append(explanation)
      allInds.append(class_ind)

  allImg = torch.cat(allImg,dim=0)
  allExpl = torch.cat(allExpl,dim=0)
  allInds = torch.cat(allInds,dim=0)


  deletion = Deletion()
  result_dic = deletion(model,allImg.clone(),allExpl.clone(),allInds)
  dauc_mean = result_dic["dauc"]
  dc_mean = result_dic["dc"]

  insertion = Insertion()
  result_dic = insertion(model,allImg.clone(),allExpl.clone(),allInds)
  iauc_mean = result_dic["iauc"]
  ic_mean = result_dic["ic"]


  iic_ad = IIC_AD()
  result_dic = iic_ad(model,allImg,allExpl,allInds)
  iic_mean,ad_mean = result_dic["iic"],result_dic["ad"]

  add = ADD()
  result_dic = add(model,allImg,allExpl,allInds)
  add_mean = result_dic["add"]


  result_dict = {
            "DAUC": dauc_mean
            ,"IAUC": iauc_mean
            ,"DC": dc_mean
            ,"IC":ic_mean
            ,"IIC": iic_mean
            ,"AD": ad_mean
            ,"ADD": add_mean
          }


  return result_dict

In [ ]:
model.cnn

In [ ]:
results = []
for id, layer in enumerate(model.cnn):
  if isinstance(layer, torch.nn.Conv2d):
    result_dic = get_GradCAMscore(images, labels, model, id)
    results.append((id, result_dic))

In [ ]:
for id, result_dic in results:
  print(id, result_dic)

In [ ]:
model.cnn
